In [ ]:
import numpy as np
import matplotlib.pylab as plt
import sys
import word2vec
import os

In [ ]:
wordvecs_dir = os.environ["WORDVECS_DIR"]
dataset_dir = os.environ["DATASET_DIR"]
octave_eval_dir = os.environ["OCTAVE_EVAL_DIR"]

In [ ]:
w2v_vector_file = os.path.join(wordvecs_dir, "text9Bvin.bin")
w2v_model = word2vec.load(w2v_vector_file)

In [ ]:
def construct_W(w2v_model, vocab):
    """
    Creates and returns a numpy ndarray of dimensions (200 x size(vocab)).
    The nth column in this array represents the Word2Vec vector for the nth word in vocab.
    Words in vocab not in the vocabulary for w2v_model will be ignored/skipped.
    
    :param w2v_model: A Word2Vec model from the word2vec package (available in pypi)
    :param vocab: A list of words
    :return: returns a numpy ndarray
    """
    return np.asarray([w2v_model.get_vector(word) for word in vocab if word in w2v_model.vocab], dtype=np.float16).T

In [ ]:
def get_invalid_labels(labels, valid_labels):
    """
    Returns a dictionary whose keys are words in labels that are not in valid_labels.
    The values of the dictionary are the corresponding indices of the invalid words in labels.
    
    :param labels: A list of labels
    :param valid_labels: A list of valid labels.
    :return: return a dictionary
    """
    return {word: idx for (idx, word) in enumerate(labels) if word not in valid_labels}

In [ ]:
def scale(arr):
    return (arr - arr.min()) / (arr.max() - arr.min())

In [ ]:
def nonlinearity(arr, coef=-1, offset=1, power=2, alpha=0.005):
    return np.exp(coef*np.power((arr-offset),power)/ alpha)

In [ ]:
from oct2py import octave
octave.addpath(octave_eval_dir)

In [ ]:
%matplotlib inline

---

# Load data

In [ ]:
feature_dir = os.path.join(dataset_dir, "alexnet-feats")
data = np.load(os.path.join(feature_dir, "iaprtc_alexfc7.npz"))
with open(os.path.join(feature_dir, "iaprtc_dictionary.txt")) as dictionary:
    D = dictionary.read().splitlines()
with open(os.path.join(feature_dir, "iaprtc_trainlist.txt")) as trainlist:
    train_ims = [ im.split('/')[-1] for im in trainlist.read().splitlines() ]
    train_ims_full = [ im for im in trainlist.read().splitlines() ]
with open(os.path.join(feature_dir, "iaprtc_testlist.txt")) as testlist:
    test_ims_full = [ im for im in testlist.read().splitlines() ]

xTr = data['xTr'].T # xtrain
yTr = data['yTr'].T # ytrain
xTe = data['xTe'].T # xtest
yTe = data['yTe'].T # ytest

In [ ]:
print xTr.shape
print yTr.shape
print xTe.shape
print yTe.shape

---

# Filter out labels not in W2V vocabulary

In [ ]:
to_remove = get_invalid_labels(D, w2v_model.vocab)

In [ ]:
D = filter(lambda word: word not in to_remove.keys(), D)

In [ ]:
yTr = np.delete(yTr, to_remove.values(), axis=1)

In [ ]:
yTe = np.delete(yTe, to_remove.values(), axis=1)

In [ ]:
print yTr.shape
print yTe.shape

In [ ]:
orig_yTr = yTr
orig_yTe = yTe

---

# Construct label-to-w2v_vocab correlation matrix

In [ ]:
W_L = construct_W(w2v_model, D)

In [ ]:
W_V = np.load(os.path.join(wordvecs_dir, "w_v_16.npy")) #construct_W(w2v_model, w2v_model.vocab)
#np.save(os.path.join(wordvecs_dir, "w_v_16.npy"), W_V)

---

# Expand vocabulary

In [ ]:
expansion_arr = np.load(os.path.join(wordvecs_dir, "expansion_arr.npy")) #np.dot(W_V.T, W_L)
#np.save(os.path.join(wordvecs_dir, "expansion_arr.npy"), expansion_arr)

In [ ]:
expanded = np.dot(yTr, nonlinearity(expansion_arr).T) #np.load(os.path.join(wordvecs_dir, "expanded.npy"))
#np.save(os.path.join(wordvecs_dir, "expanded.npy"), expanded)

In [ ]:
# Randomly select an image
i=np.random.randint(0, yTr.shape[1])

# Run example
imname = os.path.join(dataset_dir, "images", "images", "%s.jpg" % train_ims_full[i])
print "Looking at the "+str(i)+"th image: "+imname
im=plt.imread(imname)

# Prediction
#threshold = np.percentile(yHat[i], 99)
#top_idxs = [idx for idx in yHat[i].argsort()[::-1] if yHat[i][idx] > threshold]
#ypwords = [d_words[idx] for idx in top_idxs]
ypwords = [w2v_model.vocab[idx] for idx in expanded[i].argsort()[::-1][:10]]
#ypwords = [D[j] for j in yHat[i].argsort()[::-1] [ 0:(yHat[i]>0.2).sum() ] ]
# Truth
ytwords = [D[idx] for idx in np.nonzero(orig_yTr[i])[0]]
#ytwords = [D[j] for j in np.where(yTe[i] > 0.5)[0] ]
plt.imshow(im)

print 'Predicted: '+ ', '.join(ypwords)
print 'Truth:     '+ ', '.join(ytwords)

plt.figure()